In [1]:
import pandas as pd
import numpy as np
from lr.text_processing.util import pre_process_nli_df

In [2]:
df = pd.read_csv("data/toy/train.csv")
df.head(2)

,premise,hypothesis,label
0,A couple playing with a little boy on the beach.,A couple are playing with a young child outside.,entailment
1,A man with a red shirt is watching another man...,The men are outside.,neutral


In [3]:
pre_process_nli_df(df)
df.head(2)

,premise,hypothesis,label
0,a couple playing with a little boy on the beach,a couple are playing with a young child outside,entailment
1,a man with a red shirt is watching another man...,the men are outside,neutral
